In [ ]:
# scikit-learnのバーションは 0.19の必要があります
!pip install scikit-learn==0.19

# scikit-learn modelをWatson MLに登録

## 認証情報の設定

In [ ]:
wml_credentials={
    # 事前準備した認証情報を張り付けて下さい
}

## 分類モデルの生成

In [ ]:
# 必要ライブラリのロード
from IPython.display import display
import pandas as pd
from sklearn.datasets import load_iris

In [ ]:
# 学習データの準備
iris = load_iris()
columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
df_iris = pd.DataFrame(iris.data, columns=columns)
target = iris.target
display(df_iris.head())

In [ ]:
# Modelの生成
# 注意　モデルは確率値を返すものである必要があります。SVMではうまくいきません。
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(df_iris, target)

In [ ]:
# テスト用データの準備
values = df_iris.iloc[[0,1,50,51,100,101]].values.tolist()
print(values)

In [ ]:
# テストの実施 
# 予測値は [0, 0, 1, 1, 2, 2]になっているはずです
lr.predict(values)

## Watson ML Model / Webサービスの登録

In [ ]:
# Watson ML Clientの生成
from watson_machine_learning_client import WatsonMachineLearningAPIClient
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# Modelの登録
model_details = client.repository.store_model( lr, "scikit-learn lr model" )

In [ ]:
# Webサービスの登録
model_id = model_details["metadata"]["guid"]
deployment_details = client.deployments.create( artifact_uid=model_id, name="scikit-learn lr deployment" )

In [ ]:
# Webサービスのテスト
model_endpoint_url = client.deployments.get_scoring_url( deployment_details )
print(model_endpoint_url)
client.deployments.score( model_endpoint_url, { "values" : values } )